# LIVRIA : Système de recommandation par filtrage colllaboratif

   Ce jupyter notebook contient le code permettant de prédire les thèmes de livres susceptibles de plaire à l'utilisateur.

   Nous utiliserons dans un premier temps notre base de données issue d'un questionnaire dont vous pouvez observer les résultats dans le fichier answerLivria.csv. Ce dataset est axé sur les thèmes de prédilection des utilisateurs. Nous avons déjà nettoyé et vectorisé ce set de donnée (cf. dataCleaning.ipynb) et analysé les réponses (cf. dataVizualisation.ipynb).
   Dans un second temps, nous utiliserons la base de données Goodbooks-10k, qui a l'avantage de réunir beaucoup plus de réponses et qui comprend notamment les notes attribuées par les utilisateurs aux livres qu'ils ont lus. 
   
* Le set de données de Goodbooks-10 :
http://fastml.com/goodbooks-10k-a-new-dataset-for-book-recommendations/

Vous pourrez retrouver le notebook jupyter dédié au collaborative filtering pour les livres notés de Goodbooks-10k dans le fichier Livria_Goodbooks10k_Collaborative_filtering.ipynb

## Import des librairies

On commence par importer les librairies utilisées dans ce notebook.

In [358]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

# Plot data
import matplotlib.pyplot as plt
%matplotlib inline

## Import des données

Vous pouvez retrouver l'ensemble des données utilisées dans le dossier './data'


In [359]:
# Lecture du fichier 'df_sortie.csv' contenant les thèmes choisis  et vectorisés des réponses du questionnaire.
dataTheme = pd.read_csv('df_sortie.csv', sep='\t')
# On supprime la colonne inutile :
del dataTheme['Unnamed: 0']

In [360]:
# On montre les premières lignes de dataTheme
dataTheme.head()

,RomanFiction,BdComics,ArtsCulture,DocMedia,Erotisme,Esoterisme,SanteBE,HistGeo,Jeunesse,LittEtrangere,ScienceTechnique,LoisirVie,SHS,Philosophie
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,1,1,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,0,0,0,1,0,0,0,0,0,0


In [361]:
print('Les dimensions de dataTheme sont de : ' + str(dataTheme.shape))

Les dimensions de dataTheme sont de : (1279, 14)


## Création des sets d'entraînement et de test

On sépare le dataset en deux set distincts : un pour l'entraînement de notre modèle de prédiction et un pour tester ce modèle. On garde 25% des données pour le set de test.

In [362]:
train_data_theme, test_data_theme = train_test_split(dataTheme, test_size=0.25)

## Filtrage collaboratif

Nous allons prendre en considération deux modèle pour le filtrage collaboratif, le "memory-based" et le "model-based".

D'abord, on crée une matrice utilisateur-thème pour l'entraînement des données.

In [363]:
train_data_theme_matrix = np.zeros((1279,14))
for line in train_data_theme.itertuples():
    train_data_theme_matrix[line[0], :] = line[1:]

On crée ensuite une matrice utilisateur-thème pour tester le modèle.

In [364]:
test_data_theme_matrix = np.zeros((1279,14))
for line in test_data_theme.itertuples():
    test_data_theme_matrix[line[0], :] = line[1:]

## Filtrage collaboratif avec la méthode Memory-Based 

L'idée sous-jacente derrière le modèle dit "**memory-based**" est de calculer et d'utiliser les **similarités** entre utilisateurs et/ou items -ici les thèmes- et d'utiliser ces facteurs comme des "poids"  permettant la prédiction d'un thème, d'une note attribuée à un livre, ou autre. 

Nous allons tester les deux types de filtrage collaboratif:

* Item-Item 
* Utilisateur-Item 

Nous utilisons le coéfficient de similarité. Pour cela, nous importons la fonction "pairwise_distances" de sklearn. 

On calcule d'abord la similarité entre les utilisateurs.

In [365]:
from sklearn.metrics import pairwise
user_similarity_theme = pairwise.cosine_similarity(train_data_theme_matrix)

In [366]:
user_similarity_theme[:5, 0:5]

array([[1.        , 0.        , 0.40824829, 0.70710678, 0.40824829],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.40824829, 0.        , 1.        , 0.57735027, 0.        ],
       [0.70710678, 0.        , 0.57735027, 1.        , 0.        ],
       [0.40824829, 0.        , 0.        , 0.        , 1.        ]])

Calcul de la similarité entre les thèmes :

In [367]:
item_similarity_theme = pairwise.cosine_similarity(train_data_theme_matrix.T)

In [368]:
item_similarity_theme[:5, 0:5]

array([[1.        , 0.56381179, 0.33166971, 0.35786502, 0.3181418 ],
       [0.56381179, 1.        , 0.00516811, 0.15480849, 0.19006578],
       [0.33166971, 0.00516811, 1.        , 0.28705467, 0.11094004],
       [0.35786502, 0.15480849, 0.28705467, 1.        , 0.07552632],
       [0.3181418 , 0.19006578, 0.11094004, 0.07552632, 1.        ]])

On définit une méthode pour réaliser les prédictions. 

In [369]:
def predict(choices, similarity, kind='user'):
    
    sum_sim = np.array([np.abs(similarity).sum(axis=1)])
    sum_sim[sum_sim == 0] = 1    
    if kind == 'user':
        return similarity.dot(choices) / sum_sim.T
    elif kind == 'item':
        return choices.dot(similarity) / sum_sim

Cette méthode permet de prédire les thèmes susceptibles d'intéresser un utilisateur. Soit elle prend en considération les thèmes qui lui plaisent déjà, soit elle regarde les thèmes de prédilection d'autres utilisateurs ayant donné des réponses similaires.

In [370]:
item_prediction_theme = predict(train_data_theme_matrix, item_similarity_theme, 'item')

In [371]:
item_prediction_theme[0:5,0:3]

array([[0.25899107, 0.37554162, 0.09203992],
       [0.        , 0.        , 0.        ],
       [0.34291062, 0.31434597, 0.19591343],
       [0.16561524, 0.1353966 , 0.09062775],
       [0.22308328, 0.3348789 , 0.15081012]])

In [372]:
user_prediction_theme = predict(train_data_theme_matrix, user_similarity_theme, 'user')
user_prediction_theme[0:5,0:3]

array([[0.97855045, 0.46710068, 0.07221207],
       [0.        , 0.        , 0.        ],
       [0.98606546, 0.32010754, 0.1023586 ],
       [1.        , 0.29485011, 0.09423576],
       [0.90381968, 0.64430962, 0.10059428]])

On mesure la performance du modèle avec le calcul de la RMSE (root-mean-square error), c'est-à-dire la racine carrée de l'erreur quadratique. Cette méthode compare les vraies réponses aux réponses prédites par notre modèle.

In [373]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, true_value):
    prediction = prediction.flatten()
    true_value = true_value.flatten()
    return sqrt(mean_squared_error(prediction, true_value))

RMSE pour la prédiction basée sur la comparaison entre les utilisateurs. 

In [374]:
user_CF_RMSE_theme = rmse(user_prediction_theme, test_data_theme_matrix)
print('RMSE basée sur les utilisateurs : ', user_CF_RMSE_theme)

RMSE basée sur les utilisateurs :  0.3740093820934641


In [387]:
item_CF_RMSE_theme = rmse(item_prediction_theme, test_data_theme_matrix)
print('RMSE basée sur les thèmes : ', item_CF_RMSE_theme)

RMSE basée sur les thèmes :  0.3573586382578174


## Filtrage collaboratif avec la méthode Model-based

La même logique développée dans la partie précédente (cf. I.1 memory-based collaborative filtering) peut être utilisée dans la méthode dite "model-based" : les similarités entre utilisateurs et/ou items peuvent être calculées et associées à un *modèle*, et on peut ensuite utiliser ce modèle pour faire nos prédictions. 

Le filtrage collaboratif dit "model-based" repose sur la factorisation de matrice. 

Nous allons utiliser un algorithme "SVD-based" permettant de réduire les dimensions de notre set de données et de guarder les caractéristiques principales, c'est-à-dire déterminantes de nos prédictions.

Check the sparsity of our dataset.

In [376]:
sparsity_theme=round(1.0-len(dataTheme)/float(1279*14),3)
print('The sparsity level of dataTheme is ' +  str(sparsity_theme*100) + '%')

The sparsity level of dataTheme is 92.9%


Decompose the train_data_theme_matrix using the SVD method.

In [377]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_theme_matrix, k = 13)

Create the diagonale matrix.

In [378]:
s_diag_matrix=np.diag(s)

Compute the rating predictions from the decomposition values.

In [379]:
X_pred_theme = np.dot(np.dot(u, s_diag_matrix), vt)

Compute the model RMSE.

In [380]:
for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
    u, s, vt = svds(train_data_theme_matrix, k = k)
    s_diag_matrix=np.diag(s)
    X_pred_theme = np.dot(np.dot(u, s_diag_matrix), vt)
    print('SVD-based CF RMSE (k={}): {}'.format(k, str(rmse(X_pred_theme, test_data_theme_matrix))))

SVD-based CF RMSE (k=1): 0.37501837562341744
SVD-based CF RMSE (k=2): 0.39537583550113853
SVD-based CF RMSE (k=3): 0.41031562053890486
SVD-based CF RMSE (k=4): 0.42240731792621994
SVD-based CF RMSE (k=5): 0.432333958049813
SVD-based CF RMSE (k=6): 0.44028735431214905
SVD-based CF RMSE (k=7): 0.4477493940110138
SVD-based CF RMSE (k=8): 0.4547903224322869
SVD-based CF RMSE (k=9): 0.46126874907961063
SVD-based CF RMSE (k=10): 0.46619820019591574
SVD-based CF RMSE (k=11): 0.4707318376777333
SVD-based CF RMSE (k=12): 0.4745585468509398
SVD-based CF RMSE (k=13): 0.4777774873107353
